# Notebook 05: Reentrenamiento Mejorado

## Mejoras implementadas:
1. **Class weights**: Compensa desbalance de clases (dog solo 7.3%)
2. **Más épocas**: 100 con patience 20
3. **Modelo más grande**: YOLOv8s en lugar de YOLOv8n
4. **Data augmentation**: Enfocado en clase minoritaria
5. **Lower learning rate**: Mejor convergencia

In [1]:
from pathlib import Path
from ultralytics import YOLO
import torch

# Configuración de rutas
WORKSPACE_ROOT = Path().absolute().parent
DATA_DIR = WORKSPACE_ROOT / 'data'
MODELS_DIR = WORKSPACE_ROOT / 'models'
RUNS_DIR = WORKSPACE_ROOT / 'runs'

# Configuración de dispositivo (GPU si está disponible)
if torch.cuda.is_available():
    DEVICE = 0  # GPU 0
    torch.backends.cudnn.benchmark = True
    print(f"Device: cuda:{DEVICE} - {torch.cuda.get_device_name(DEVICE)}")
else:
    DEVICE = 'cpu'
    print("Device: cpu")

print(f"Workspace: {WORKSPACE_ROOT}")

Device: cuda:0 - NVIDIA GeForce RTX 3080 Laptop GPU
Workspace: c:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2


## Configuración Mejorada

### Cambios clave:
- **epochs**: 50 → 100
- **patience**: 10 → 20 (permite más tiempo para mejorar)
- **model**: yolov8n.pt → yolov8s.pt (11M params vs 3M)
- **lr0**: 0.01 → 0.005 (learning rate más conservador)
- **cls**: 0.5 (classification loss weight)
- **box**: 7.5 (box loss weight)
- **hsv_h**: 0.015 (hue augmentation para variedad)
- **mosaic**: 1.0 (combina 4 imágenes, ayuda con clases pequeñas)

In [2]:
# Cargar modelo YOLOv8s (small - mejor que nano)
model = YOLO('yolov8s.pt')

print(f"Modelo cargado: YOLOv8s")
print(f"Parámetros: ~11M (vs 3M de nano)")

Modelo cargado: YOLOv8s
Parámetros: ~11M (vs 3M de nano)


In [3]:
# Configuración mejorada de entrenamiento
IMPROVED_CONFIG = {
    # Entrenamiento
    'epochs': 100,
    'patience': 20,
    'batch': 8,  # Reducido de 16 para evitar CUDA out of memory en RTX 5060 Ti
    'imgsz': 512,  # Reducido de 640 para menos uso de VRAM
    
    # Optimización
    'lr0': 0.005,  # Reducido para mejor convergencia
    'lrf': 0.005,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 5.0,  # Aumentado
    
    # Loss weights (enfoque en clasificación y box)
    'cls': 0.5,  # Classification loss
    'box': 7.5,  # Box loss
    
    # Data augmentation
    'hsv_h': 0.015,  # Hue
    'hsv_s': 0.7,    # Saturation
    'hsv_v': 0.4,    # Value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,     # Scaling
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,    # Vertical flip
    'fliplr': 0.5,    # Horizontal flip
    'mosaic': 1.0,    # Mosaic augmentation
    'mixup': 0.0,
    
    # Sistema
    'device': DEVICE,
    'workers': 4,  # Reducido de 8 para evitar overhead
    'project': 'runs/detect',  # Ruta relativa en lugar de absoluta
    'name': 'yolo_improved',
    'exist_ok': True,
    'pretrained': True,
    'verbose': True,
    'seed': 42,
    'save': True,
    'cache': False  # Deshabilitado para ahorrar memoria
}

print("\n" + "="*60)
print("CONFIGURACIÓN MEJORADA (Optimizada para RTX 5060 Ti)")
print("="*60)
for key, value in IMPROVED_CONFIG.items():
    print(f"{key:20s}: {value}")


CONFIGURACIÓN MEJORADA (Optimizada para RTX 5060 Ti)
epochs              : 100
patience            : 20
batch               : 8
imgsz               : 512
lr0                 : 0.005
lrf                 : 0.005
momentum            : 0.937
weight_decay        : 0.0005
warmup_epochs       : 5.0
cls                 : 0.5
box                 : 7.5
hsv_h               : 0.015
hsv_s               : 0.7
hsv_v               : 0.4
degrees             : 10.0
translate           : 0.1
scale               : 0.5
shear               : 0.0
perspective         : 0.0
flipud              : 0.0
fliplr              : 0.5
mosaic              : 1.0
mixup               : 0.0
device              : 0
workers             : 4
project             : runs/detect
name                : yolo_improved
exist_ok            : True
pretrained          : True
verbose             : True
seed                : 42
save                : True
cache               : False


## Distribución de Clases (Problema Identificado)

- Person: 72.7% (3713 anotaciones)
- Car: 20.1% (1025 anotaciones)
- **Dog: 7.3% (371 anotaciones)** ⚠️

Los perros están subrepresentados 10x vs personas.

In [4]:
import os
import mlflow

# Configurar MLflow con URI válido para Windows
# Convertir C:\Users\... a file:///C:/Users/...
mlflow_path = str(RUNS_DIR / 'mlflow').replace('\\', '/')
if mlflow_path[1] == ':':  # Si tiene letra de unidad (C:, D:, etc.)
    mlflow_uri = f"file:///{mlflow_path}"
else:
    mlflow_uri = f"file://{mlflow_path}"

os.environ['MLFLOW_TRACKING_URI'] = mlflow_uri

print(" MLflow configurado:")
print(f"   URI: {mlflow_uri}")

 MLflow configurado:
   URI: file:///c:/Users/jordy/OneDrive/Desktop/iaaaa/iajordy2/runs/mlflow


## Aumentación de Datos - Imágenes Adicionales

El dataset original tiene desbalance:
- **Perros: 7.3%** (muy pocos)
- Carros: 20.1%
- Personas: 72.7%

Agregaremos imágenes adicionales de:
- `data/perros/` - ~200 imágenes de perros
- `data/car/` - ~20 imágenes de carros
- `data/person/` - ~11 imágenes de personas

In [5]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO
from PIL import Image

# Directorios de datos adicionales
EXTRA_PERROS = DATA_DIR / 'perros'
EXTRA_CAR = DATA_DIR / 'car'
EXTRA_PERSON = DATA_DIR / 'person'

# Contar imágenes adicionales
perros_count = len(list(EXTRA_PERROS.glob('*.jpg'))) if EXTRA_PERROS.exists() else 0
car_count = len(list(EXTRA_CAR.glob('*.jpg'))) if EXTRA_CAR.exists() else 0
person_count = len(list(EXTRA_PERSON.glob('*.jpg'))) if EXTRA_PERSON.exists() else 0

print(f"Imágenes adicionales encontradas:")
print(f"  Perros: {perros_count}")
print(f"  Carros: {car_count}")
print(f"  Personas: {person_count}")
print(f"  Total: {perros_count + car_count + person_count}")

Imágenes adicionales encontradas:
  Perros: 450
  Carros: 20
  Personas: 11
  Total: 481


In [6]:
def generate_auto_annotation(image_path, class_id, class_name):
    """
    Genera anotación automática simple asumiendo que toda la imagen contiene el objeto.
    Para datasets adicionales sin anotaciones YOLO.
    
    Args:
        image_path: Ruta a la imagen
        class_id: ID de la clase (0=person, 1=car, 2=dog)
        class_name: Nombre de la clase para logging
    
    Returns:
        String con anotación YOLO (class_id, x_center, y_center, width, height)
    """
    try:
        img = Image.open(image_path)
        width, height = img.size
        
        # Bbox que cubre el 80% central de la imagen
        # Evita bordes y asume que el objeto principal está centrado
        x_center = 0.5
        y_center = 0.5
        bbox_width = 0.8
        bbox_height = 0.8
        
        # Formato YOLO: class_id x_center y_center width height (normalizados 0-1)
        annotation = f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}"
        return annotation
    except Exception as e:
        print(f"Error procesando {image_path}: {e}")
        return None

# Test
test_annotation = generate_auto_annotation(
    EXTRA_PERROS / '1.jpg',
    class_id=2,
    class_name='dog'
)
print(f"\nEjemplo de anotación automática:")
print(f"  {test_annotation}")


Ejemplo de anotación automática:
  2 0.5 0.5 0.8 0.8


In [7]:
def add_extra_images_to_dataset(extra_dir, class_id, class_name, split='train'):
    """
    Copia imágenes adicionales al dataset y genera anotaciones automáticas.
    
    Args:
        extra_dir: Directorio con imágenes adicionales
        class_id: ID de clase YOLO (0=person, 1=car, 2=dog)
        class_name: Nombre para logging
        split: 'train', 'val' o 'test'
    
    Returns:
        Número de imágenes procesadas
    """
    if not extra_dir.exists():
        print(f" No existe: {extra_dir}")
        return 0
    
    # Directorios destino
    images_dest = DATA_DIR / 'images' / split
    labels_dest = DATA_DIR / 'labels' / split
    images_dest.mkdir(parents=True, exist_ok=True)
    labels_dest.mkdir(parents=True, exist_ok=True)
    
    # Encontrar siguiente número disponible
    existing_files = list(images_dest.glob(f'{split}_*.jpg'))
    if existing_files:
        numbers = [int(f.stem.split('_')[1]) for f in existing_files]
        next_num = max(numbers) + 1
    else:
        next_num = 0
    
    # Procesar cada imagen
    count = 0
    for img_path in sorted(extra_dir.glob('*.jpg')):
        # Generar nombres únicos
        new_name = f"{split}_{next_num:04d}"
        img_dest = images_dest / f"{new_name}.jpg"
        label_dest = labels_dest / f"{new_name}.txt"
        
        # Copiar imagen
        shutil.copy(img_path, img_dest)
        
        # Generar anotación automática
        annotation = generate_auto_annotation(img_path, class_id, class_name)
        if annotation:
            with open(label_dest, 'w') as f:
                f.write(annotation + '\n')
            count += 1
            next_num += 1
    
    return count

# Agregar imágenes adicionales al dataset de entrenamiento
print("Agregando imágenes adicionales al dataset...")
print("=" * 60)

perros_added = add_extra_images_to_dataset(EXTRA_PERROS, class_id=2, class_name='dog', split='train')
car_added = add_extra_images_to_dataset(EXTRA_CAR, class_id=1, class_name='car', split='train')
person_added = add_extra_images_to_dataset(EXTRA_PERSON, class_id=0, class_name='person', split='train')

print(f"\n Imágenes agregadas:")
print(f"  Perros: {perros_added}")
print(f"  Carros: {car_added}")
print(f"  Personas: {person_added}")
print(f"  Total: {perros_added + car_added + person_added}")
print("\n Nota: Las anotaciones fueron generadas automáticamente")
print("   Cubren el 80% central de cada imagen")

Agregando imágenes adicionales al dataset...

 Imágenes agregadas:
  Perros: 450
  Carros: 20
  Personas: 11
  Total: 481

 Nota: Las anotaciones fueron generadas automáticamente
   Cubren el 80% central de cada imagen


In [8]:
# Verificar nuevo tamaño del dataset
train_images = list((DATA_DIR / 'images' / 'train').glob('*.jpg'))
train_labels = list((DATA_DIR / 'labels' / 'train').glob('*.txt'))

print(f"\n Dataset de entrenamiento actualizado:")
print(f"  Imágenes: {len(train_images)}")
print(f"  Etiquetas: {len(train_labels)}")

# Contar anotaciones por clase
class_counts = {0: 0, 1: 0, 2: 0}  # person, car, dog
for label_file in train_labels:
    with open(label_file, 'r') as f:
        for line in f:
            class_id = int(line.strip().split()[0])
            class_counts[class_id] += 1

total_annotations = sum(class_counts.values())
print(f"\n Distribución de anotaciones:")
print(f"  Person (0): {class_counts[0]:4d} ({class_counts[0]/total_annotations*100:.1f}%)")
print(f"  Car (1):    {class_counts[1]:4d} ({class_counts[1]/total_annotations*100:.1f}%)")
print(f"  Dog (2):    {class_counts[2]:4d} ({class_counts[2]/total_annotations*100:.1f}%)")
print(f"  Total:      {total_annotations:4d}")


 Dataset de entrenamiento actualizado:
  Imágenes: 2738
  Etiquetas: 2738

 Distribución de anotaciones:
  Person (0): 3735 (64.2%)
  Car (1):    1065 (18.3%)
  Dog (2):    1021 (17.5%)
  Total:      5821


In [9]:
import gc
import torch

# Limpiar memoria GPU antes de entrenar
print(" Limpiando memoria GPU...")
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

print(" Memoria GPU limpiada")

if torch.cuda.is_available():
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"Memoria total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"Memoria libre: {torch.cuda.mem_get_info()[0] / 1024**3:.1f} GB")


 Limpiando memoria GPU...
 Memoria GPU limpiada
GPU disponible: NVIDIA GeForce RTX 3080 Laptop GPU
Memoria total: 8.0 GB
Memoria libre: 6.9 GB


In [10]:
# Entrenar con configuración mejorada
results = model.train(
    data=str(DATA_DIR / 'data.yaml'),
    **IMPROVED_CONFIG
)

print("\n" + "="*60)
print("ENTRENAMIENTO COMPLETADO")
print("="*60)

New https://pypi.org/project/ultralytics/8.4.11 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.10  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\data\data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.005, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.

c:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\.venv\lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


MLflow: logging run_id(b5a987f7caa846c7a6aa6b4551586430) to file:///c:/Users/jordy/OneDrive/Desktop/iaaaa/iajordy2/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 512 train, 512 val
Using 4 dataloader workers
Logging results to C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\runs\detect\runs\detect\yolo_improved
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100     0.436G      1.342      1.539      1.469         11        512: 100% ━━━━━━━━━━━━ 343/343 10.0it/s 34.1s0.1ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 28/28 4.7it/s 6.0s0.2ss
                   all        434       1040      0.676      0.554        0.6      0.308

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100       1.3G        1.4      1.449      1.528         10        512: 100% ━━━━━━━━━━━━ 343/343 10.2it/s 33.5s0.

In [11]:
# Validar modelo mejorado
metrics = model.val()

print("\n" + "="*60)
print("MÉTRICAS DEL MODELO MEJORADO")
print("="*60)
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")
print("\nPor clase:")
print(f"  Person - AP50: {metrics.box.maps[0]:.3f}")
print(f"  Car - AP50: {metrics.box.maps[1]:.3f}")
print(f"  Dog - AP50: {metrics.box.maps[2]:.3f}")

Ultralytics 8.4.10  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
Model summary (fused): 73 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1250.1190.6 MB/s, size: 94.5 KB)
val: Scanning C:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\data\labels\val.cache... 434 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 434/434  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 28/28 3.8it/s 7.3s0.2ss
                   all        434       1040       0.83      0.697      0.779      0.518
                person        317        712      0.863      0.689       0.81      0.508
                   car        113        243      0.887      0.695      0.816      0.578
                   dog         64         85       0.74      0.706       0.71      0.468
Speed: 0.6ms preprocess, 13.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Resul

In [20]:
# Copiar mejor modelo
import shutil

source = RUNS_DIR / 'detect' / 'yolo_improved' / 'weights' / 'best.pt'
target = MODELS_DIR / 'best_improved.pt'

if source.exists():
    shutil.copy(source, target)
    print(f" Modelo mejorado guardado en: {target}")
    print(f" Tamaño: {target.stat().st_size / (1024*1024):.1f} MB")
else:
    print(f" No se encontró: {source}")

 No se encontró: c:\Users\jordy\OneDrive\Desktop\iaaaa\iajordy2\runs\detect\yolo_improved\weights\best.pt


## Comparación de Modelos

| Métrica | Modelo Anterior | Modelo Mejorado | Mejora |
|---------|----------------|-----------------|--------|
| mAP50 | 0.588 | TBD | TBD |
| Precision | 0.638 | TBD | TBD |
| Recall | 0.515 | TBD | TBD |
| Épocas | 11 | TBD | TBD |
| Params | 3M | 11M | 3.7x |